In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geoband.API import *
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopandas import GeoDataFrame
# GetCompasData로 주피터 노트북 경로에 파일 저장
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

%matplotlib inline

import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot

from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
plt.rc('font', family='NanumBarunGothic') #한글 폰트 지정

import mapboxgl
mapboxgl.__version__
from mapboxgl.viz import *
import os
from mapboxgl.utils import create_color_stops
from mapboxgl.utils import create_numeric_stops
import numpy as np

# 시각화 방향성
- Polygon 시각화(Grid포함)
- Point 시각화
- Heatmap 시각화
- ?

> ## Polygon 시각화
- Pydeck 보단 Mapboxgl 가 범주 표시에 있어서 더 좋음
- Pydeck 의 경우 Layer 를 겹칠 수 있다는 장점

1) Mapboxgl Polygon 단위 시각화
2) Pydeck Polygon 단위 시각화

- mapbox://styles/mapbox/streets-v11
- mapbox://styles/mapbox/outdoors-v11
- mapbox://styles/mapbox/light-v10
- mapbox://styles/mapbox/dark-v10
- mapbox://styles/mapbox/satellite-v9
- mapbox://styles/mapbox/satellite-streets-v11
- mapbox://styles/mapbox/navigation-preview-day-v4
- mapbox://styles/mapbox/navigation-preview-night-v4
- mapbox://styles/mapbox/navigation-guidance-day-v4
- mapbox://styles/mapbox/navigation-guidance-night-v4

### 1) Mapboxgl Polygon 시각화

In [2]:
def mapboxgl_polygon_visual(data,region,token_API,col,bin_lst,visual_3D):
    # data : geometry와 시각화 할 value가 같이 담긴 (Geo)DataFrame이 입력값
    # region(str) : 서울, 수원, 창원, 진주
    # token_API(str) : mapbox Api key
    # visual_3D(bool) : True or False - 3D로 나타낼 것인지
    # col(str) : 어떤 column에 대해 visualize 할 것인지
    # bin_lst(lst) : 해당 columns의 값을 어떤 구간으로 나눌 것인지
    data_copy = data.copy()
    data_copy = gpd.GeoDataFrame(data_copy,geometry = "geometry").to_crs(epsg="4326")

    data_copy.to_file("test.geojson",driver="GeoJSON")
    with open("test.geojson") as f:
        data_copy = json.loads(f.read())
    # geojson 으로 내보내고 json으로 읽어오기
        
    if region == "수원":
        center = [127.008,37.280]
    elif region == "서울":
        center = [126.986, 37.565]
        
    color_breaks = bin_lst
    color_stops = create_color_stops(color_breaks, colors='BuPu')
    # 시각화 값에 따른 색상의 범주 지정
        
    viz = ChoroplethViz(access_token=token_API, data = data_copy, color_property= col, 
                        color_stops=color_stops, center=center, zoom=11, style = "mapbox://styles/mapbox/dark-v10")

    if not visual_3D:
        viz.show()
    else:
        viz.bearing = -15 ; viz.pitch = 45 # bearing : 좌우, pitch : 상하 회전
        
        # 각 데이터에 'col'를 기준으로 height 값을 줍니다.
        viz.height_property = col

        # 높이 값의 범주를 0 ~ 3000 사이로 매핑
        viz.height_stops = create_numeric_stops(bin_lst, 0, 3000)
        viz.height_function_type = 'interpolate'

        # render again
        viz.show()        

In [3]:
token = 'pk.eyJ1Ijoia2lteXVzaW50d28iLCJhIjoiY2t1cnRlNzd4MG1sZjJubG53MHh5MDVtMSJ9.GGjIXQbBt9faPTRoO6bTIw'

데이터 불러오기 1

In [4]:
test = gpd.read_file("2.수원시_112신고(격자별 신고횟수).geojson")

In [ ]:
# np_linspace = np.linspace(start=0, stop=5000, num=5) 0 부터 5000 사이의 값을 5 등분
np_linspace = [0,10,20,100,1000,2000,3000,4000]

mapboxgl_polygon_visual(test,"수원",
                        token,
                        "rep_cnt",
                        np_linspace,
                        False)

데이터 불러오기 2

In [6]:
dd = gpd.read_file("TL_SCCO_SIG.shp",encoding="cp949")

suwon_SIG = dd[dd["SIG_KOR_NM"].str.contains("수원")]
rep_cnt = [100, 2000, 50, 5]
suwon_SIG["rep_cnt"] = rep_cnt

In [7]:
# np_linspace = np.linspace(start=0, stop=5000, num=5) 0 부터 5000 사이의 값을 5 등분
np_linspace = [0,10,20,100,1000,2000,3000,4000]

mapboxgl_polygon_visual(suwon_SIG,"수원",
                        token,
                        "rep_cnt",
                        np_linspace,
                        True)

### 2) Pydeck GeoJson 시각화

- map_style = pdk.map_styles.LIGHT
- map_style = pdk.map_styles.DARK
- map_style = pdk.map_styles.DARK_NO_LABELS
- map_style = pdk.map_styles.LIGHT_NO_LABELS

In [9]:
# 수원시 격자별 신고횟수
data = gpd.read_file("2.수원시_112신고(격자별 신고횟수).geojson")
data.head(3)

,gid,rep_cnt,geometry
0,다사49aa19ab,0,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ..."
1,다사49aa19ba,0,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ..."
2,다사49aa20aa,0,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ..."


In [10]:
data["정규화"] = data["rep_cnt"] + 1 / data["rep_cnt"].max() 

In [15]:
import pydeck as pdk
def pydeck_polygon_visual(data,region,token_API,col,visual_3D):

    # data : geometry와 시각화 할 value가 같이 담긴 GeoDataFrame이 입력값
    # region(str) : 서울, 수원, 창원, 진주
    # token_API(str) : mapbox Api key
    # visual_3D(bool) : True or False - 3D로 나타낼 것인지
    # col(str) : 어떤 column에 대해 visualize 할 것인지
    
    # geojson 파일 -> json
    data.to_file("test.geojson",driver="GeoJSON")
    
    # json으로 읽어오기
    with open("test.geojson") as f:
        df = json.loads(f.read())
    
    if region == "수원":
        center = [127.008,37.280]
    elif region == "서울":
        center = [126.986, 37.565]
        
    val = "properties."+col
    # properties.사건발생건수 -> 이런 형식
        
    if visual_3D:
        VIEW = pdk.ViewState(latitude= center[1], longitude= center[0],
          zoom=10, max_zoom=16,
          pitch=45, # 45도 상하회전
          bearing=-15 ) # 좌우회전
        
        layers = [pdk.Layer("GeoJsonLayer", data=df, stroked = False, # Outline
                get_line_color=[255, 0, 255],filled=True,pickable=True,
                auto_highlight=True, extruded=True, # 3차원 하면 True
                get_elevation= val, # 어떤 값을 이용해 높이를 부여할 것인지
                get_fill_color="[0,255*"+"properties.정규화"+",0]",wireframe = False ) ] # Wireframe : 폴리곤 간에 구분선
               # get_fill_color에서 색상 설정해주기 
        r= pdk.Deck(
            layers,
            initial_view_state=VIEW,
            map_style = pdk.map_styles.LIGHT,
            mapbox_key = token,
        )
        return r.to_html()
    else:
        VIEW = pdk.ViewState(latitude= center[1],longitude= center[0],zoom=10, max_zoom=16)
        
        layers = [pdk.Layer("GeoJsonLayer", data=df, stroked = True, get_line_color= [255, 0, 255],filled=True, pickable=True, 
                    auto_highlight=True, extruded=False, get_fill_color="[0,255*"+"properties.정규화"+",0]",wireframe = True ) ]
        r= pdk.Deck(layers,initial_view_state=VIEW, map_style = pdk.map_styles.LIGHT, mapbox_key = token)
        return r.to_html()
        

In [ ]:
pydeck_polygon_visual(data,"수원",token,"rep_cnt",visual_3D = True)

In [ ]:
pydeck_polygon_visual(data,"수원",token,"rep_cnt",visual_3D = False)

> ## Point 시각화

### 1) Mapboxgl Point 시각화

In [29]:
from fiona.crs import from_string

def mapboxgl_point_visual(data,region,token_API,col,bin_lst,visual_color):
    # data : geometry와 시각화 할 value가 같이 담긴 (Geo)DataFrame이 입력값
    # region(str) : 서울, 수원, 창원, 진주
    # token_API(str) : mapbox Api key
    # visual_color(bool) : True or False - color로 나타낼 것인지
    # col(str) : 어떤 column에 대해 visualize 할 것인지
    # bin_lst(lst) : 해당 columns의 값을 어떤 구간으로 나눌 것인지
    data_copy = data.copy()
    # 소문자로 열 이름 변경 LAT -> lat
    data_copy.columns = [i.lower() for i in data_copy.columns]
    
    if "geometry" in data_copy.columns:
        data_copy = gpd.GeoDataFrame(data_copy,geometry = "geometry").to_crs(epsg="4326")
    else:
        epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
        data_copy['Point'] = data_copy.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
        data_copy = gpd.GeoDataFrame(data_copy, geometry='Point', crs = epsg4326)
        
    data_copy.to_file("test.geojson",driver="GeoJSON")
    with open("test.geojson") as f:
        data_copy = json.loads(f.read())
    # geojson 으로 내보내고 json으로 읽어오기
        
    if region == "수원":
        center = [127.008,37.280]
    elif region == "서울":
        center = [126.986, 37.565]
   
    viz = CircleViz(access_token=token_API, data = data_copy, center=center, zoom=11, style = "mapbox://styles/mapbox/light-v10")
    
    # 색상 부여하기
    if visual_color:
        viz.color_property = col
        viz.color_stops = create_color_stops(bin_lst, colors='BuPu')
        viz.show()
    else:
        viz.show()

In [ ]:
tt = pd.read_csv("20.수원시_여성안심택배함현황.csv")
tt["rep_cnt"] = 1 # 개수 관련 열만 추가해주기

mapboxgl_point_visual(tt,"수원",token,"rep_cnt",[0,5],visual_color=False)
# mapboxgl로 하면 경계가 안 나옴.

----------------------------------------------------------------------------------------------------------------

### 2) Pydeck ScatterPlot 시각화

In [17]:
test10 = pd.read_csv("20.수원시_여성안심택배함현황.csv")
test10.head(2)

,fac_nm,sido_nm,sgg_nm,sgg_cd,road_addr,jibun_addr,lat,lon,week_start_time,week_end_time,...,c_box_type,c_box_cnt,c_box_depth,c_box_width,c_box_height,install_date,phone_num,manage_agency,manage_agency_pn,base_date
0,파장동 행정복지센터,경기도,수원시,41110,경기도 수원시 장안구 파장천로 60 (파장동),NaN,37.310320,126.997419,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
1,율천동 행정복지센터,경기도,수원시,41110,경기도 수원시 장안구 서부로 2106번길 27 (율천동),NaN,37.297615,126.971425,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19


In [18]:
# 수원시 exterior
# suwon_police_area.geometry.unary_union.exterior 생성법
exterior_data = gpd.read_file("2.수원시_112신고(격자별 신고횟수).geojson")
exterior_data = gpd.read_file("수원시_exterior.geojson")

In [25]:
import shapely
import pydeck
def pydeck_scatterplot_visual(data,exterior,region,radius):
    # data : 위도 경도 정보가 담긴 dataframe
    # exterior_data : 경계 정보가 담긴 data - 수원시 전체 Polygon
    # radious : scatterplot radius
    # region : 수원 서울 창원 진주
    exterior_data = exterior.copy()
    # Pydeck에 맞는 데이터 형식으로 변환 : Polygon이냐 MultiPolygon이냐에 따라 다름
    
    if type(exterior_data.loc[0,"geometry"]) == shapely.geometry.polygon.Polygon:
        exterior_data["coordinates"] = exterior_data["geometry"].apply(lambda polygon : list(polygon.exterior.coords))
        del exterior_data['geometry']
        exterior_data = pd.DataFrame(exterior_data)
        
    elif type(exterior_data.loc[0,"geometry"]) == shapely.geometry.multipolygon.MultiPolygon:
        def multipolygon_to_coordinates(x):
            lon, lat = x[0].exterior.xy
            return [[x, y] for x, y in zip(lon, lat)]
        exterior_data['coordinates'] = exterior_data['geometry'].apply(multipolygon_to_coordinates)

        del exterior_data['geometry']
        # Pydeck은 geoDataFrame 말고 dataframe 인식
        exterior_data = pd.DataFrame(exterior_data)    

    df = data.copy()
    # 소문자로 열 이름 변경 LAT -> lat
    df.columns = [i.lower() for i in df.columns]    

    polygon = pydeck.Layer(
        'PolygonLayer',
        exterior_data,
        stroked=False,
        get_polygon='coordinates', # 데이터 형식 [[],[],[]]
        #get_polygon='-',
        get_fill_color=[0, 0, 0, 20]
    )
    
    layer = pdk.Layer(
    'ScatterplotLayer',
    data,
    get_position='[lon, lat]',
    get_radius=radius,
    get_fill_color='[0, 255, 255]',
    pickable=True,
    auto_highlight=True
    )
    
    if region == "수원":
        center = [127.008,37.280]
    elif region == "서울":
        center = [126.986, 37.565]    

    view_state = pdk.ViewState(
        longitude=center[0],
        latitude=center[1],
        zoom=10)

    r = pdk.Deck(layers=[layer,polygon], initial_view_state=view_state,mapbox_key = token,map_style = pdk.map_styles.LIGHT)
    return r.to_html()

In [26]:
pydeck_scatterplot_visual(test10,exterior_data,"수원",50)

> ## Heatmap 시각화

In [27]:
import shapely
import pydeck as pdk
import pydeck
def pydeck_heatmap_visual(data,exterior,region):
    # data : 위도 경도 정보가 담긴 dataframe
    # exterior_data : 경계 정보가 담긴 data - 수원시 전체 Polygon
    # radious : scatterplot radius
    # region : 수원 서울 창원 진주

    # Pydeck에 맞는 데이터 형식으로 변환 : Polygon이냐 MultiPolygon이냐에 따라 다름
    exterior_data = exterior.copy()
    if type(exterior_data.loc[0,"geometry"]) == shapely.geometry.polygon.Polygon:
        exterior_data["coordinates"] = exterior_data["geometry"].apply(lambda polygon : list(polygon.exterior.coords))
        del exterior_data['geometry']
        exterior_data = pd.DataFrame(exterior_data)
        
    elif type(exterior_data.loc[0,"geometry"]) == shapely.geometry.multipolygon.MultiPolygon:
        def multipolygon_to_coordinates(x):
            lon, lat = x[0].exterior.xy
            return [[x, y] for x, y in zip(lon, lat)]
        exterior_data['coordinates'] = exterior_data['geometry'].apply(multipolygon_to_coordinates)

        del exterior_data['geometry']
        # Pydeck은 geoDataFrame 말고 dataframe 인식
        exterior_data = pd.DataFrame(exterior_data)    

    df = data.copy()
    # 소문자로 열 이름 변경 LAT -> lat
    df.columns = [i.lower() for i in df.columns]    

    polygon = pdk.Layer(
        'PolygonLayer',
        exterior_data,
        stroked=False,
        get_polygon='coordinates', # 데이터 형식 [[],[],[]]
        #get_polygon='-',
        get_fill_color=[0, 0, 0, 20]
    )
    
    layer = pdk.Layer(
    'HeatmapLayer',
    data,
    get_position='[lon, lat]',
    )
    
    if region == "수원":
        center = [127.008,37.280]
    elif region == "서울":
        center = [126.986, 37.565]    

    view_state = pdk.ViewState(
        longitude=center[0],
        latitude=center[1],
        zoom=10)

    r = pdk.Deck(layers=[layer,polygon], initial_view_state=view_state,mapbox_key = token,map_style = pdk.map_styles.LIGHT)
    return r.to_html()

In [ ]:
pydeck_heatmap_visual(test10,exterior_data,"수원")

###########################################################################################################################

In [80]:
# Pydeck에 맞는 데이터 형식으로 변환
def multipolygon_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

data['lat'] = data['geometry'].apply(lambda coord: coord.y)
data['lon'] = data['geometry'].apply(lambda coord: coord.x)

data['coordinates'] = data['geometry'].apply(multipolygon_to_coordinates)

del data['geometry']

# Pydeck은 geoDataFrame 말고 dataframe 인식
data = pd.DataFrame(data)
data["rep_cnt_scaled"] = data["rep_cnt"] / data["rep_cnt"].max()

AttributeError: 'MultiPolygon' object has no attribute 'y'

In [ ]:
layer = pdk.Layer(
    'CPUGridLayer', # 사용할 Layer 타입
    data, # 시각화에 쓰일 데이터프레임
    get_position=["lat","lon"], # geometry 정보를 담고있는 컬럼 이름
    #get_fill_color='[0, 255*rep_cnt_scaled, 0]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    pickable=True, # 지도와 interactive 한 동작 on
    auto_highlight=True # 마우스 오버(hover) 시 박스 출력
)

# Set the viewport location
center = center = [127.0080933988626,37.28041656451134]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state,mapbox_key = token)
r.to_html()

In [ ]:
layer.extruded = True;
layer.get_elevation = 'rep_cnt';
layer.elevation_scale = 3

view_state.bearing=15
view_state.pitch=45

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

### GridLayer

In [337]:
import pydeck as pdk
import pandas as pd

CPU_GRID_LAYER_DATA = (
    "https://raw.githubusercontent.com/uber-common/" "deck.gl-data/master/website/sf-bike-parking.json"
)
df = pd.read_json(CPU_GRID_LAYER_DATA)
# 데이터 불러오기

In [338]:
df

,ADDRESS,RACKS,SPACES,COORDINATES
0,939 ELLIS ST,2,4,"[-122.42177834, 37.78346622]"
1,1380 HOWARD ST,1,2,"[-122.414411, 37.774458]"
2,1195 OAK ST,1,2,"[-122.438887, 37.772737]"
3,1387 VALENCIA ST,1,2,"[-122.42019976, 37.75087429]"
4,180 TOWNSEND ST,1,2,"[-122.392606, 37.779369]"
...,...,...,...,...
2515,900 VALENCIA ST,1,2,"[-122.42152025, 37.75825474]"
2516,91 WALTER ST,1,2,"[-122.432136, 37.767817]"
2517,916 KEARNY,1,2,"[-122.40502124, 37.79653486]"
2518,939 EDDY ST,1,2,"[-122.423112, 37.782316]"


In [15]:
# Define a layer to display on a map

layer = pdk.Layer(
    "GridLayer", df, pickable=True, extruded=True, cell_size=250, elevation_scale=4, get_position="COORDINATES",
)

view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=45)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, mapbox_key = token ,
             tooltip={"text": "{position}\nCount: {count}"},map_style = pdk.map_styles.LIGHT)
r.to_html()

In [372]:
# 수원시 격자별 신고횟수
data = gpd.read_file("2.수원시_112신고(격자별 신고횟수).geojson")
data.head(3)

# Pydeck에 맞는 데이터 형식으로 변환
def multipolygon_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

data['coordinates'] = data['geometry'].apply(multipolygon_to_coordinates)

data['centroid'] = data['geometry'].apply(lambda x: x.centroid)

data['centroid'] = data['centroid'].apply(lambda coord: [coord.x,coord.y])
del data["geometry"]
del data["coordinates"]
data = pd.DataFrame(data)

In [377]:
data.head(2)

,gid,rep_cnt,geometry,coordinates,centroid
0,다사49aa19ab,0,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...","[[126.92476861910082, 37.27076573094027], [126...","[126.92616988433635, 37.27189926537194]"
1,다사49aa19ba,0,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ...","[[126.92475147461319, 37.27301912088102], [126...","[126.9261527809103, 37.27415265543407]"


In [378]:
test10 = pd.read_csv("20.수원시_여성안심택배함현황.csv")
test10

,fac_nm,sido_nm,sgg_nm,sgg_cd,road_addr,jibun_addr,lat,lon,week_start_time,week_end_time,...,c_box_type,c_box_cnt,c_box_depth,c_box_width,c_box_height,install_date,phone_num,manage_agency,manage_agency_pn,base_date
0,파장동 행정복지센터,경기도,수원시,41110,경기도 수원시 장안구 파장천로 60 (파장동),NaN,37.310320,126.997419,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
1,율천동 행정복지센터,경기도,수원시,41110,경기도 수원시 장안구 서부로 2106번길 27 (율천동),NaN,37.297615,126.971425,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
2,정자1동 행정복지센터,경기도,수원시,41110,경기도 수원시 장안구 정자로 130번길 20 (정자동),NaN,37.302214,126.991292,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
3,조원1동 행정복지센터,경기도,수원시,41110,경기도 수원시 장안구 금당로 56 (조원동),NaN,37.301778,127.016499,0:00,23:59,...,중형/02+중소형/03,02/3+03/12,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
4,연무동 행정복지센터,경기도,수원시,41110,경기도 수원시 장안구 경수대로 757 (연무동),NaN,37.290560,127.021531,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
5,평동 행정복지센터,경기도,수원시,41110,경기도 수원시 권선구 평동로 76번길 13 (평동),NaN,37.258059,126.994170,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
6,권선1동 행정복지센터,경기도,수원시,41110,경기도 수원시 권선구 경수대로 352번길 29 (권선동),NaN,37.257750,127.024653,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
7,곡선동 행정복지센터,경기도,수원시,41110,경기도 수원시 권선구 동수원로 146번길 174 (곡반정동),NaN,37.243317,127.033345,0:00,23:59,...,중형/02+중소형/03,02/3+03/12,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
8,버드내도서관,경기도,수원시,41110,경기도 수원시 권선구 세권로94번길 29(세류동),NaN,37.256822,127.014776,0:00,23:59,...,중형/02+중소형/03,02/3+03/12,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19
9,서호노인복지관,경기도,수원시,41110,경기도 수원시 권선구 구운로 4번길 34 (구운동),NaN,37.278496,126.976190,0:00,23:59,...,중형/02+중소형/03,02/4+03/16,02/55+03/55,02/45+03/45,02/60+03/30,2017-01-01,1899-4711,수원시,031-228-3218,2021-01-19


In [382]:
# Define a layer to display on a map

layers = pdk.Layer(
    "CPUGridLayer", test10, pickable=True, extruded=False, elevation_scale=3, get_position=["lon","lat"],
)

view_state = pdk.ViewState(latitude=37.280, longitude=127.008, zoom=11, bearing=15, pitch=45)

# Render
r = pdk.Deck(layers=[layers], initial_view_state=view_state, mapbox_key = token ,
             #tooltip={"text": "{position}\nCount: {count}"}
             map_style = pdk.map_styles.LIGHT)
r.to_html()



### GeoJson Layer

In [163]:
# 수원시 격자별 신고횟수
data = gpd.read_file("2.수원시_112신고(격자별 신고횟수).geojson")
data.head(3)

# Pydeck에 맞는 데이터 형식으로 변환
def multipolygon_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

data['coordinates'] = data['geometry'].apply(multipolygon_to_coordinates)
del data["geometry"]
data = pd.DataFrame(data)

In [376]:
data.head(3)

,gid,rep_cnt,geometry,coordinates,centroid
0,다사49aa19ab,0,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...","[[126.92476861910082, 37.27076573094027], [126...","[126.92616988433635, 37.27189926537194]"
1,다사49aa19ba,0,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ...","[[126.92475147461319, 37.27301912088102], [126...","[126.9261527809103, 37.27415265543407]"
2,다사49aa20aa,0,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ...","[[126.92471717991359, 37.27752589816265], [126...","[126.92611856834758, 37.27865943295857]"


In [ ]:
pd.read_csv

In [231]:
# 데이터는 json 형태로 geojson (x)
import json
with open("2.수원시_112신고(격자별 신고횟수).geojson") as f:
    data = json.loads(f.read())
#data_df = pd.DataFrame.from_dict(data["features"])
# geojson -> dict -> dataframe

In [166]:
with open("2.수원시_112신고(격자별 신고횟수).geojson") as f:
    data = json.loads(f.read())
    
# json으로 읽어와야지 나타나네 왜???

In [228]:
DATA_URL = 'https://raw.githubusercontent.com/MinnPost/simple-map-d3/master/example-data/world-population.geo.json'

world = gpd.read_file(DATA_URL)

In [167]:
INITIAL_VIEW_STATE = pdk.ViewState(
  latitude= 37.280,
  longitude= 127.008,
  zoom=10,
  #max_zoom=16,
  pitch=120, # 상하회전
  bearing=0 # 좌우회전
)

layers = [
    pdk.Layer(
        "GeoJsonLayer",
        data=data,
        stroked=False,
        get_line_color=[255, 0, 255],
        filled=True,
        pickable=True,
        auto_highlight=True,
        extruded=True,
        get_elevation='rep_cnt',
        get_fill_color='[255, 255, rep_cnt * 255]'),
]

pdk.Deck(
    layers,
    initial_view_state=INITIAL_VIEW_STATE,
    map_style = pdk.map_styles.DARK,
    mapbox_key = token,
    tooltip={'text': '{gid}, 사건발생건수 : {rep_cnt}'},
).to_html()

In [279]:
INITIAL_VIEW_STATE = pdk.ViewState(
  latitude= 37.280,
  longitude= 127.008,
  zoom=10,
  #max_zoom=16,
  pitch=45,
  bearing=15
)

#polygon = pydeck.Layer(
#    'PolygonLayer',
#    LAND_COVER,
#    stroked=False,
    # processes the data as a flat longitude-latitude pair
#    get_polygon='-',
#    get_fill_color=[0, 0, 0, 20]
#)

geojson = pdk.Layer(
    'GeoJsonLayer',
    data,
    opacity=0.8,
    stroked=False, # No Outline
    filled=True, # Poylgon Should be Filled
    extruded=True, # 3차원 하면 True 
    #wireframe=True,
    get_elevation='properties.rep_cnt',
    get_fill_color='[255, 255, rep_cnt * 255]',
    get_line_color=[255, 255, 255],
    pickable=True
)

r = pdk.Deck(
    layers=[geojson],
    initial_view_state=INITIAL_VIEW_STATE,
    mapbox_key = token,
    map_style = pdk.map_styles.DARK_NO_LABELS)


- map_style = pdk.map_styles.LIGHT
- map_style = pdk.map_styles.DARK
- map_style = pdk.map_styles.DARK_NO_LABELS
- map_style = pdk.map_styles.LIGHT_NO_LABELS

In [113]:
import pydeck as pdk

DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=49.254,
  longitude=-123.13,
  zoom=11,
  max_zoom=16,
  pitch=45,
  bearing=0
)

polygon = pydeck.Layer(
    'PolygonLayer',
    LAND_COVER,
    stroked=True,
    # processes the data as a flat longitude-latitude pair
    get_polygon='-',
    get_fill_color=[0, 0, 0, 20]
)

geojson = pydeck.Layer(
    'GeoJsonLayer',
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation='properties.valuePerSqm / 20',
    get_fill_color='[255, 255, properties.growth * 255]',
    get_line_color=[255, 255, 255],
    pickable=True
)

r = pydeck.Deck(
    layers=[geojson],
    initial_view_state=INITIAL_VIEW_STATE,
    map_style = pdk.map_styles.LIGHT,
    mapbox_key = token)

r.to_html()

In [25]:
# 데이터는 json 형태로 geojson (x)
import json
with open("2.수원시_112신고(격자별 신고횟수).geojson") as f:
    data = json.loads(f.read())